In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import plotly.graph_objects as go
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import matplotlib as plt
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
df = pd.read_csv('2021-06-metropolitan-stop-and-search.csv')
df.columns

Index(['Type', 'Date', 'Part of a policing operation', 'Policing operation',
       'Latitude', 'Longitude', 'Gender', 'Age range',
       'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation',
       'Object of search', 'Outcome', 'Outcome linked to object of search',
       'Removal of more than just outer clothing'],
      dtype='object')

In [3]:
def criminality(df):
    if df['Outcome'] == "A no further action disposal" or df['Outcome'] == "Caution (simple or conditional)":
        return 0
    else:
        return 1
    
df['criminality'] = df.apply(criminality, axis=1)

In [4]:
def ethnicity(df):
    et = df['Self-defined ethnicity']
    if "White" in et and (not 'Mixed' in et):
        return 'White'
    elif "Black" in et and (not 'Mixed' in et):
        return 'Black'
    elif "Asian" in et and (not 'Mixed' in et):
        return 'Asian'
    elif 'Mixed' in et:
        return 'Mixed'
    elif et == "Other ethnic group - Any other ethnic group":
        return 'Other'
    else:
        return 'Not stated'

df['Self-defined ethnicity'].fillna('Other ethnic group - Not stated', inplace=True)
df['ethnicity'] = df.apply(ethnicity, axis=1)

In [5]:
counts = df['ethnicity'].value_counts()
ethnicities = pd.DataFrame({'ethnicity':counts.index, 'total':counts.values})
counts_nocrime =  df[df['criminality'] == 0]['ethnicity'].value_counts()
counts_crime = df[df['criminality'] == 1]['ethnicity'].value_counts()
ethnicities['crime'] = counts_crime.values
ethnicities['nocrime'] = counts_nocrime.values
ethnicities

,ethnicity,total,crime,nocrime
0,White,5215,1559,3656
1,Not stated,4324,1148,3496
2,Black,4067,828,2919
3,Asian,2023,586,1437
4,Mixed,580,197,383
5,Other,476,144,332


In [6]:
ethnicities['percentage_crime'] = ethnicities['crime'] / ethnicities['total'] 
ethnicities['percentage_nocrime'] = ethnicities['nocrime'] / ethnicities['total'] 
ethnicities

,ethnicity,total,crime,nocrime,percentage_crime,percentage_nocrime
0,White,5215,1559,3656,0.298945,0.701055
1,Not stated,4324,1148,3496,0.265495,0.808511
2,Black,4067,828,2919,0.203590,0.717728
3,Asian,2023,586,1437,0.289669,0.710331
4,Mixed,580,197,383,0.339655,0.660345
5,Other,476,144,332,0.302521,0.697479


In [7]:
df['Gender'].fillna('Not stated', inplace=True)
counts = df['Gender'].value_counts()
genders = pd.DataFrame({'gender':counts.index, 'total':counts.values})
counts_nocrime =  df[df['criminality'] == 0]['Gender'].value_counts()
counts_crime = df[df['criminality'] == 1]['Gender'].value_counts().reindex(df['Gender'].unique(), fill_value=0)
genders['crime'] = counts_crime.values
genders['nocrime'] = counts_nocrime.values
genders['percentage_crime'] = genders['crime'] / genders['total'] 
genders['percentage_nocrime'] = genders['nocrime'] / genders['total'] 
genders

,gender,total,crime,nocrime,percentage_crime,percentage_nocrime
0,Male,15182,4048,11134,0.266632,0.733368
1,Female,1285,332,953,0.258366,0.741634
2,Not stated,213,82,131,0.384977,0.615023
3,Other,5,0,5,0.000000,1.000000


In [8]:
df['Age range'].fillna('Not stated', inplace=True)
counts = df['Age range'].value_counts()
ages = pd.DataFrame({'age':counts.index, 'total':counts.values})
counts_nocrime =  df[df['criminality'] == 0]['Age range'].value_counts()
counts_crime = df[df['criminality'] == 1]['Age range'].value_counts()
ages['crime'] = counts_crime.values
ages['nocrime'] = counts_nocrime.values
ages['percentage_crime'] = ages['crime'] / ages['total'] 
ages['percentage_nocrime'] = ages['nocrime'] / ages['total'] 
ages

,age,total,crime,nocrime,percentage_crime,percentage_nocrime
0,18-24,5282,1677,3605,0.317493,0.682507
1,25-34,3632,1262,2370,0.347467,0.652533
2,over 34,3146,883,2263,0.280674,0.719326
3,10-17,2422,461,2023,0.190339,0.835260
4,Not stated,2201,178,1961,0.080872,0.890959
5,under 10,2,1,1,0.500000,0.500000


In [9]:
trace1 = go.Bar(
    x=ethnicities['ethnicity'],
    y=ethnicities['crime'],
    text=ethnicities['percentage_crime'].apply(lambda x: '{percent:.2%}'.format(percent=x)),
    name='Crime',
    marker_color="#E45756",
)

trace2 = go.Bar(
    x=ethnicities['ethnicity'],
    y=ethnicities['nocrime'],
    text=ethnicities['percentage_nocrime'].apply(lambda x: '{percent:.2%}'.format(percent=x)),
    name='Non-crime',
    marker_color="#54A54B"
)

trace3 = go.Bar(
    x=ethnicities['ethnicity'],
    y=ethnicities['total'],
    name='Total',
    marker_color="#4C78A8"
)
layout = go.Layout(
    updatemenus=[
        dict(
            type="dropdown",
            active=0,
            xanchor="right",
            yanchor="top",
            direction="down",
            buttons=list([
                dict(label="Summary",
                     method="update",
                     args=[{"visible": [True, True, True]
                     },
                     {"title": "Summary of stop and searches by ethnicity",
                     }]),
                dict(label="Crime",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Criminal officer judgment stop and searches by ethnicity"}]),
                dict(label="Non-crime",
                     method="update",
                     args=[{"visible": [False, True, False]},
                     {"title": "Non-criminal officer judgment top and searches with by ethnicity"}]),
                dict(label="Total",
                     method="update",
                     args=[{"visible": [False, False, True]
                     },
                     {"title": "Total stop and searches by ethnicity",
                     }])
            ]),
        )
    ])

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

fig.update_layout(
    xaxis_title="Ethnicity",
    yaxis_title="Number of People",
    title="Summary of stop and searches by ethnicity"
)
fig.show()

In [10]:
trace1 = go.Bar(
    x=genders['gender'],
    y=genders['crime'],
    text=genders['percentage_crime'].apply(lambda x: '{percent:.2%}'.format(percent=x)),
    name='Crime',
    marker_color="#E45756",
)

trace2 = go.Bar(
    x=genders['gender'],
    y=genders['nocrime'],
    text=genders['percentage_nocrime'].apply(lambda x: '{percent:.2%}'.format(percent=x)),
    name='Non-crime',
    marker_color="#54A54B"
)

trace3 = go.Bar(
    x=genders['gender'],
    y=genders['total'],
    name='Total',
    marker_color="#4C78A8"
)
layout = go.Layout(
    updatemenus=[
        dict(
            type="dropdown",
            active=0,
            xanchor="right",
            yanchor="top",
            direction="down",
            buttons=list([
                dict(label="Summary",
                     method="update",
                     args=[{"visible": [True, True, True]
                     },
                     {"title": "Summary of stop and searches by gender",
                     }]),
                dict(label="Crime",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Criminal officer judgment stop and searches by gender"}]),
                dict(label="Non-crime",
                     method="update",
                     args=[{"visible": [False, True, False]},
                     {"title": "Non-criminal officer judgment top and searches with by gender"}]),
                dict(label="Total",
                     method="update",
                     args=[{"visible": [False, False, True]
                     },
                     {"title": "Total stop and searches by gender",
                     }])
            ]),
        )
    ])

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

fig.update_layout(
    xaxis_title="Gender",
    yaxis_title="Number of People",
    title="Summary of stop and searches by gender"
)
fig.show()

In [11]:
trace1 = go.Bar(
    x=ages['age'],
    y=ages['crime'],
    text=ages['percentage_crime'].apply(lambda x: '{percent:.2%}'.format(percent=x)),
    name='Crime',
    marker_color="#E45756",
)

trace2 = go.Bar(
    x=ages['age'],
    y=ages['nocrime'],
    text=ages['percentage_nocrime'].apply(lambda x: '{percent:.2%}'.format(percent=x)),
    name='Non-crime',
    marker_color="#54A54B"
)

trace3 = go.Bar(
    x=ages['age'],
    y=ages['total'],
    name='Total',
    marker_color="#4C78A8"
)
layout = go.Layout(
    updatemenus=[
        dict(
            type="dropdown",
            active=0,
            xanchor="right",
            yanchor="top",
            direction="down",
            buttons=list([
                dict(label="Summary",
                     method="update",
                     args=[{"visible": [True, True, True]
                     },
                     {"title": "Summary of stop and searches by age",
                     }]),
                dict(label="Crime",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Criminal officer judgment stop and searches by age"}]),
                dict(label="Non-crime",
                     method="update",
                     args=[{"visible": [False, True, False]},
                     {"title": "Non-criminal officer judgment top and searches with by age"}]),
                dict(label="Total",
                     method="update",
                     args=[{"visible": [False, False, True]
                     },
                     {"title": "Total stop and searches by age",
                     }])
            ]),
        )
    ])

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

fig.update_layout(
    xaxis_title="Age range",
    yaxis_title="Number of People",
    title="Summary of stop and searches by age"
)
fig.show()

In [13]:
df["Date"] = pd.to_datetime(df["Date"])
df["weekday"] = df["Date"].dt.weekday
df['date'] = df['Date'].dt.date

In [14]:
date_counts = df['date'].value_counts()
dates = pd.DataFrame({'date':date_counts.index, 'total':date_counts.values})
counts_nocrime =  df[df['criminality'] == 0]['date'].value_counts()
counts_crime = df[df['criminality'] == 1]['date'].value_counts()
dates['crime'] = counts_crime.values
dates['nocrime'] = counts_nocrime.values
dates

,date,total,crime,nocrime
0,2021-06-11,734,206,541
1,2021-06-05,679,197,504
2,2021-06-25,666,195,483
3,2021-06-24,662,193,478
4,2021-06-04,648,174,473
5,2021-06-19,634,171,470
6,2021-06-09,626,171,469
7,2021-06-03,623,165,455
8,2021-06-10,620,158,455
9,2021-06-23,614,157,449


In [15]:
trace1 = go.Bar(
    x=dates['date'],
    y=dates['crime'],
    name='Crime',
    marker_color="#E45756",
)

trace2 = go.Bar(
    x=dates['date'],
    y=dates['nocrime'],
    name='Non-crime',
    marker_color="#54A54B"
)

trace3 = go.Bar(
    x=dates['date'],
    y=dates['total'],
    name='Total',
    marker_color="#4C78A8"
)
layout = go.Layout(
    updatemenus=[
        dict(
            type="dropdown",
            active=0,
            xanchor="right",
            yanchor="top",
            direction="down",
            buttons=list([
                dict(label="Summary",
                     method="update",
                     args=[{"visible": [True, True, True]
                     },
                     {"title": "Summary of stop and searches through time",
                     }]),
                dict(label="Crime",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Criminal officer judgment stop and searches through time"}]),
                dict(label="Non-crime",
                     method="update",
                     args=[{"visible": [False, True, False]},
                     {"title": "Non-criminal officer judgment top and searches through time"}]),
                dict(label="Total",
                     method="update",
                     args=[{"visible": [False, False, True]
                     },
                     {"title": "Total stop and searches through time",
                     }])
            ]),
        )
    ])

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Number of Stop and Searches",
    title="Summary of stop and searches through time"
)
fig.show()

In [16]:
weekday_counts = df['weekday'].value_counts()
weekdays = pd.DataFrame({'weekday':weekday_counts.index, 'total':weekday_counts.values})
counts_nocrime =  df[df['criminality'] == 0]['weekday'].value_counts()
counts_crime = df[df['criminality'] == 1]['weekday'].value_counts()
weekdays['crime'] = counts_crime.values
weekdays['nocrime'] = counts_nocrime.values
weekdays

,weekday,total,crime,nocrime
0,2,2870,789,2081
1,1,2600,712,1915
2,4,2594,679,1888
3,3,2489,665,1836
4,5,2395,653,1730
5,0,2014,512,1502
6,6,1723,452,1271


In [21]:
trace1 = go.Bar(
    x=weekdays['weekday'],
    y=weekdays['crime'],
    name='Crime',
    marker_color="#E45756",
)

trace2 = go.Bar(
    x=weekdays['weekday'],
    y=weekdays['nocrime'],
    name='Non-crime',
    marker_color="#54A54B"
)

trace3 = go.Bar(
    x=weekdays['weekday'],
    y=weekdays['total'],
    name='Total',
    marker_color="#4C78A8"
)
layout = go.Layout(
    updatemenus=[
        dict(
            type="dropdown",
            active=0,
            xanchor="right",
            yanchor="top",
            direction="down",
            buttons=list([
                dict(label="Summary",
                     method="update",
                     args=[{"visible": [True, True, True]
                     },
                     {"title": "Summary of stop and searches through time",
                     }]),
                dict(label="Crime",
                     method="update",
                     args=[{"visible": [True, False, False]},
                           {"title": "Criminal officer judgment stop and searches through time"}]),
                dict(label="Non-crime",
                     method="update",
                     args=[{"visible": [False, True, False]},
                     {"title": "Non-criminal officer judgment top and searches through time"}]),
                dict(label="Total",
                     method="update",
                     args=[{"visible": [False, False, True]
                     },
                     {"title": "Total stop and searches through time",
                     }])
            ]),
        )
    ])

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

fig.update_layout(
    xaxis_title="Weekday",
    yaxis_title="Number of Stop and Searches",
    title="Summary of stop and searches through time",   
)
fig.update_xaxes(categoryorder='array', categoryarray= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
fig.show()